In [2]:
import pandas as pd

In [ ]:
import spacy
from spacy.matcher import Matcher
import re
from typing import List, Dict, Optional

In [ ]:
"""
nlp_interpreter.py
-------------------
Lightweight NLP Interpreter using spaCy for intent and entity extraction
for maritime vessel monitoring queries.

Example:
    "Show the last known position of INS Kolkata."
    "Predict where MSC Flaminia will be after 30 minutes."
"""


class MaritimeNLPInterpreter:
    """
    A class to interpret natural language queries related to maritime vessel data.
    Extracts:
        - Intent (SHOW / PREDICT / VERIFY)
        - Vessel name
        - Time horizon (e.g., 'after 30 minutes')
        - Optional identifiers (MMSI, CallSign, IMO)
    """

    def __init__(self, vessel_list: Optional[List[str]] = None):
        """
        Initialize NLP model and rule patterns.
        Args:
            vessel_list (List[str]): Known vessel names from the database.
        """
        self.vessel_list = [v.lower() for v in vessel_list] if vessel_list else []
        self.nlp = spacy.load("en_core_web_sm")
        self.matcher = Matcher(self.nlp.vocab)
        self._register_patterns()

        # Intent keyword mapping
        self.intent_keywords = {
            "show": ["show", "display", "find", "locate", "fetch", "retrieve", "where"],
            "predict": ["predict", "forecast", "estimate", "project"],
            "verify": ["check", "validate", "verify", "compare", "confirm"],
        }

    def _register_patterns(self):
        """Register pattern rules for time expressions, MMSI, etc."""
        # Time horizon pattern
        pattern_time = [
            {"LOWER": {"IN": ["after", "in"]}},
            {"IS_DIGIT": True},
            {"LOWER": {"IN": ["minutes", "minute", "hours", "hour"]}},
        ]
        self.matcher.add("TIME_HORIZON", [pattern_time])

    # ---------------- CORE METHOD ---------------- #
    def parse_query(self, text: str) -> Dict[str, Optional[str]]:
        """
        Parse the given text and extract structured intent and entities.

        Args:
            text (str): Natural language query from user.

        Returns:
            dict: Structured info with intent, vessel_name, time_horizon, identifiers.
        """
        doc = self.nlp(text.lower())
        text_lower = text.lower()

        return {
            "intent": self._extract_intent(text_lower),
            "vessel_name": self._extract_vessel_name(text_lower),
            "time_horizon": self._extract_time_horizon(doc),
            "identifiers": self._extract_identifiers(text_lower),
        }

    # ---------------- SUBCOMPONENTS ---------------- #
    def _extract_intent(self, text_lower: str) -> Optional[str]:
        """Detect the user's intent using keyword mapping."""
        for intent, words in self.intent_keywords.items():
            if any(word in text_lower for word in words):
                return intent.upper()
        return None

    def _extract_vessel_name(self, text_lower: str) -> Optional[str]:
        """Find a vessel name by matching against known vessel list."""
        for vessel in self.vessel_list:
            if vessel in text_lower:
                # Return the properly capitalized name
                return next(v for v in self.vessel_list if v == vessel)
        return None

    def _extract_time_horizon(self, doc) -> Optional[str]:
        """Find time-related expressions like 'after 30 minutes'."""
        matches = self.matcher(doc)
        for _, start, end in matches:
            return doc[start:end].text
        return None

    def _extract_identifiers(self, text_lower: str) -> Dict[str, Optional[str]]:
        """
        Extract optional vessel identifiers such as MMSI, IMO, or CallSign.
        """
        identifiers = {}

        # MMSI (9-digit number)
        mmsi_match = re.search(r"\b\d{9}\b", text_lower)
        identifiers["mmsi"] = mmsi_match.group() if mmsi_match else None

        # IMO (starts with IMO, followed by digits)
        imo_match = re.search(r"imo\s*\d+", text_lower)
        identifiers["imo"] = (
            imo_match.group().replace("imo", "").strip() if imo_match else None
        )

        # CallSign (alphanumeric, 3–7 chars, preceded by 'callsign')
        call_match = re.search(r"callsign\s+([a-z0-9]{3,7})", text_lower)
        identifiers["call_sign"] = call_match.group(1).upper() if call_match else None

        return identifiers

In [ ]:
# from nlp_interpreter import MaritimeNLPInterpreter

# Example vessel names loaded from database or CSV
vessel_list = ["INS Kolkata", "MSC Flaminia", "Ever Given", "Maersk Alabama"]

nlp_model = MaritimeNLPInterpreter(vessel_list=vessel_list)

queries = [
    "Show the last known position of INS Kolkata.",
    "Predict where MSC Flaminia will be after 30 minutes.",
    "Check if the latest position of Ever Given is consistent with its past movement.",
    "Find vessel with MMSI 419999999.",
    "Verify CallSign AB1234 and IMO 1234567.",
]

for q in queries:
    print(f"\n🔹 Query: {q}")
    result = nlp_model.parse_query(q)
    print(result)


🔹 Query: Show the last known position of INS Kolkata.
{'intent': 'SHOW', 'vessel_name': 'ins kolkata', 'time_horizon': None, 'identifiers': {'mmsi': None, 'imo': None, 'call_sign': None}}

🔹 Query: Predict where MSC Flaminia will be after 30 minutes.
{'intent': 'PREDICT', 'vessel_name': 'msc flaminia', 'time_horizon': 'after 30 minutes', 'identifiers': {'mmsi': None, 'imo': None, 'call_sign': None}}

🔹 Query: Check if the latest position of Ever Given is consistent with its past movement.
{'intent': 'VERIFY', 'vessel_name': 'ever given', 'time_horizon': None, 'identifiers': {'mmsi': None, 'imo': None, 'call_sign': None}}

🔹 Query: Find vessel with MMSI 419999999.
{'intent': 'SHOW', 'vessel_name': None, 'time_horizon': None, 'identifiers': {'mmsi': '419999999', 'imo': None, 'call_sign': None}}

🔹 Query: Verify CallSign AB1234 and IMO 1234567.
{'intent': 'VERIFY', 'vessel_name': None, 'time_horizon': None, 'identifiers': {'mmsi': None, 'imo': '1234567', 'call_sign': 'AB1234'}}


In [ ]:
import requests

data = {
    "query": "Check if the latest position of Ever Given is consistent with its past movement."
}
r = requests.post("http://127.0.0.1:8000/interpret/", json=data)
print(r.json())

{'intent': 'VERIFY', 'vessel_name': 'ever given', 'time_horizon': None, 'identifiers': {'mmsi': None, 'imo': None, 'call_sign': None}}


In [ ]:
import pandas as pd

df1 = pd.read_pickle(
    r"F:\PyTorch_GPU\maritime_monitoring_preprocessing\processed_data\AIS_2020_01_03.pkl"
)
df1.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,CallSign,VesselType,...,IMO_IMO9835812,IMO_IMO9836397,IMO_IMO9836402,IMO_IMO9837212,IMO_IMO9840532,IMO_IMO9840685,IMO_IMO9840740,IMO_IMO9841691,IMO_IMO9849942,IMO_IMO999975000
0,18,2020-01-03 00:00:40,27.85054,-97.23628,0.0,149.7,511.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,18,2020-01-03 00:03:44,27.85055,-97.23632,0.0,186.1,511.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,18,2020-01-03 00:06:43,27.85055,-97.23629,0.0,181.6,511.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,18,2020-01-03 00:09:45,27.85055,-97.23621,0.0,175.5,511.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,18,2020-01-03 00:12:45,27.85057,-97.23626,0.0,180.3,511.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [4]:
df1.shape

(7089446, 3992)

In [5]:
df1.columns

Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'VesselName', 'CallSign', 'VesselType',
       ...
       'IMO_IMO9835812', 'IMO_IMO9836397', 'IMO_IMO9836402', 'IMO_IMO9837212',
       'IMO_IMO9840532', 'IMO_IMO9840685', 'IMO_IMO9840740', 'IMO_IMO9841691',
       'IMO_IMO9849942', 'IMO_IMO999975000'],
      dtype='object', length=3992)

In [ ]:
len(df1["VesselName"].unique())

10552

In [ ]:
df1["VesselName"].unique()[0:500]

array([nan, 'WEEKS 551', 'CHAMPAGNE CHER', 'NAUTICAST',
       'WINCHELSEA CONTROL', 'NEPTUNE TLP', 'BRUTUS TLP', 'OLYMPUS TLP',
       'LUCIUS SPAR', 'INGRAM TEST UNIT', 'T/T-SILVER CLOUD', 'CG49428',
       'US GOV UESSEL', 'HOOK N BULL', 'SD PILOT BOAT', '0.C.D',
       'LA STELLA', 'KORTRIJK', 'KALLO', 'TWINS', 'UBC TARRAGONA',
       'OCEAN PEARL', 'HUGIN EXPLORER', 'CAMPECHE BAY', 'PROGRESO',
       'ACRUX N', 'FALMOUTH', 'WARNOW DOLPHIN', 'WESTERN BOHEME',
       'WESTERN LUCREZIA', 'PACIFIC TRADER', 'SEABOARD AMERICA',
       'SV MARLIN', 'TOKYO EXPRESS', 'ENDLESS SUMMER', 'NORTHERN JULIE',
       'LULLABYE', 'RIPPLE', 'CHICAGO EXPRESS', 'TUFTY', 'REGULA',
       'VITA KOUAN', 'CMA CGM THALASSA', 'HALUNDER JET', 'UBC STOCKHOLM',
       'HYUNDAI HONGKONG', 'UBC BALBOA', 'JUAN DIEGO', 'KYPROS BRAVERY',
       'GRECO LIBERO', 'EL MATADOR', 'UBC BREMEN', 'MOTTLER',
       'LEMESSOS QUEEN', 'FOUMA', 'BABUN EXPRESS', 'MALAIKA', 'AEGEA',
       'VALROSSA', 'APOLLONIA', 'ALMA', 'CMA CG